In [1]:
import gym
import pathenv.environ
import numpy as np
import random
from collections import deque
from itertools import islice
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

/home/kolya/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
env = gym.make("TurtleBotObstEnv-v1")
env._configure(map_dir="/home/kolya/Documents/project/isyt2017rl/ROS_PROJ/learning_env/src/maps", visualize=True, its_per_map=1)

WARN: Environment '<class 'pathenv.environ.TurtleBotObstEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.98    # discount rate
        self.epsilon = 0.1  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(30, input_dim=self.state_size, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        #minibatch = random.sample(self.memory, batch_size)
        minibatch = list(islice(self.memory, 0, batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
if True:
    state_size = 640
    action_size = 4
    agent = DQNAgent(state_size, action_size)
    #agent.load("./new_reward.h5")
    done = False
    batch_size = 400
    episodes = 2000
    rewards = list()
    for e in range(episodes):
        state = env.reset()
        #print("reset done")
        state = np.reshape(state, [1, state_size])
        sesh_reward = 0
        for time in range(batch_size):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, data = env.step(action)
            if time % 100 == 0:
                print("iter:", time, "dist:", data.dist)
            reward = reward
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            sesh_reward += reward
            if done:
                #print("GOT THERE")
                break
        agent.update_target_model()
        sesh_reward /= (time + 1.)
        print("episode: {}/{}, score: {}, e: {:.2}, iters: {}"
               .format(e, episodes, sesh_reward, agent.epsilon, time))
        rewards.append(sesh_reward)
        if len(agent.memory) > batch_size:
            #print("training on memory")
            agent.replay(batch_size)
        if e % 10 == 0:
            pass
            #agent.save("./new_reward.h5")
            #print("saved model")

('iter:', 0, 'dist:', 2.998380033078686)
('iter:', 100, 'dist:', 2.536285617379822)


In [ ]:
model.save(filepath)

In [ ]:
print(rewards)